## ダミーの保険契約情報を生成する処理
入力データとして利用するのは「郵便局情報、ダミー個人情報、ダミー住所情報」の3つ

ダミー個人情報の取得元
https://tm-webtools.com/Tools/TestData

ダミー住所情報の取得元
http://jusyo.jp/

In [139]:
import pandas as pd
import numpy as np
import random
import copy

In [87]:
df_branch = pd.read_csv('data/branch_info.csv')

# 住所情報は必要最小限のデータ項目に絞り込み
df_addr = pd.read_csv('data/dummy_address_13tokyo.csv', encoding='shift-jis')
df_addr = df_addr.fillna('')
df_addr = df_addr[df_addr.address != '']
df_addr = df_addr[['postal_cd', 'prefecture', 'city', 'address']].copy()


df_person = pd.read_csv('data/dummy_personal_info_10000.csv')

In [88]:
df_branch.head(5)

,postal_area,post_office_cd,post_office_name,post_office_pronunciation
0,神田,1008,神田郵便局,かんだ
1,神田,1024,小川町駅前郵便局,おがわまちえきまえ
2,神田,1042,西神田郵便局,にしかんだ
3,神田,1077,神田北神保町郵便局,かんだきたじんぼうちょう
4,神田,1128,神田今川橋郵便局,かんだいまがわばし


In [89]:
df_person.head(3)

,last_name,first_name,last_name_pronunciation,first_name_pronunciation,gender,birthday,age,phone
0,足立,涼太,アダチ,リョウタ,男,1974/9/4,45,080-6770-4958
1,豊島,景子,テシマ,ケイコ,女,1992/1/28,28,080-4598-1444
2,柏倉,祐司,カシクラ,ユウジ,男,1987/1/30,33,080-0754-9667


In [90]:
df_addr.head(3)

,postal_cd,prefecture,city,address
3,101-8672,東京都,千代田区,三崎町２−７−１０帝都三崎町ビル
4,101-8370,東京都,千代田区,三崎町２丁目９−１８
5,101-8421,東京都,千代田区,猿楽町１−５−１５猿楽町ＳＳビル


## 生成するデータのパラメータを設定する

In [91]:
# 契約受理者の数
num_staffs = 20

# 契約受理者が所属する郵便局の数
num_post_offices = 3

# 保険契約者の数
num_contractors = 100

# 一人の契約者が保持する契約数の上限
max_contracts = 8

# 被保険者のパターン別確率（a/bは足して1以下とする必要あり）
pct_case_a = 0.5 # 契約者本人が被保険者である確率
pct_case_b = 0.4 # 家族（同一姓、同一住所）が被保険者である確率
pct_case_c = 1 - pct_case_a - pct_case_b # case_a/case_b以外は（別姓、別住所）が被保険者となる


In [92]:
# 受理者が所属する郵便局のコード一覧をセット
post_office_cds = df_branch.sample(n=num_post_offices).post_office_cd.tolist()

# 受理者一覧を生成
df_staff = df_person[df_person.age > 25]
df_staff = df_staff.sample(n=num_staffs)
df_staff.reset_index(drop=True, inplace=True)
df_staff.reset_index(inplace=True)
df_staff.rename(columns={'index': 'staff_cd'}, inplace=True)

post_office_cds = df_branch.sample(n=num_post_offices).post_office_cd.values.tolist()
df_staff['post_office_cd'] = [np.random.choice(post_office_cds) for x in range(0,num_staffs)]

In [93]:
df_staff.head(3)

,staff_cd,last_name,first_name,last_name_pronunciation,first_name_pronunciation,gender,birthday,age,phone,post_office_cd
0,0,柳,昭一,ヤナギ,ショウイチ,男,1985/9/25,34,090-3602-0717,755
1,1,栗原,昌之,クリハラ,マサユキ,男,1985/8/29,34,090-2226-4253,1128
2,2,戸川,心愛,トガワ,ココア,女,1969/10/21,50,080-8104-8561,1072


In [94]:
# 契約dictに受け取った個人情報、住所情報をセットして返却する
def set_person_info(prefix: str, person_info: pd.Series, contract_dict: dict):
    
    contract_dict ['{}_last_name'.format(prefix)] = person_info.last_name
    contract_dict ['{}_first_name'.format(prefix)] = person_info.first_name
    contract_dict ['{}_last_name_pron'.format(prefix)] = person_info.last_name_pronunciation
    contract_dict ['{}_first_name_pron'.format(prefix)] = person_info.first_name_pronunciation
    contract_dict ['{}_gender'.format(prefix)] = person_info.gender
    contract_dict ['{}_birthday'.format(prefix)] = person_info.birthday
    contract_dict ['{}_age'.format(prefix)] = person_info.age
    contract_dict ['{}_phone'.format(prefix)] = person_info.phone
        
    return contract_dict
    
def set_address(prefix: str, address_info: pd.Series, contract_dict: dict):
    
    contract_dict ['{}_postal_cd'.format(prefix)] = address_info.postal_cd
    contract_dict ['{}_address_1'.format(prefix)] = address_info.prefecture
    contract_dict ['{}_address_2'.format(prefix)] = address_info.city
    contract_dict ['{}_address_3'.format(prefix)] = address_info.address
        
    return contract_dict
    
test_dict = {}
#set_person_info('contractor', contractor, test_dict)
#set_address('contractor', contractor_address, test_dict)

In [95]:
insured_person

last_name                              神保
first_name                              悟
last_name_pronunciation               ジンポ
first_name_pronunciation              サトシ
gender                                  男
birthday                        2000/12/7
age                                    19
phone                       080-6149-8634
Name: 766, dtype: object

In [141]:
# 契約の一覧を生成
person_index = [random.randint(0,df_person.shape[0]) for x in range(0,num_contractors * max_contracts)]
addr_index = [random.randint(0,df_addr.shape[0]) for x in range(0,num_contractors * max_contracts)]

contracts = []
contract_index = 0
for i in range(0, num_contractors):
    # 保険契約者の個人情報、住所情報をランダムに選択する
    contract_dict = {}
    contractor = df_person.sample(n=1).iloc[0]
    contractor_address = df_addr.sample(n=1).iloc[0]

    # 受理者CDをセット
    contract_dict['staff_cd'] = random.randint(0, num_staffs - 1)
    
    # 契約情報をセット
    contract_dict = set_person_info('contractor', contractor, contract_dict)
    contract_dict = set_address('contractor', contractor_address, contract_dict)
        
    # 一人の契約者が保持する契約の数をランダムに決める
    num_contract = int(random.random()  ** 2 * max_contracts) + 1
    
    

    # 契約の数分ループを回す
    for j in range(0, num_contract):
        # 保険契約の種類をセット
        contract_dict['type'] = np.random.choice(['終身', '定期', '養老', '長寿支援', '特別終身'], p = [0.4, 0.2, 0.2, 0.1, 0.1])

        # 被保険者情報を3つのケースから選択する
        case_var = np.random.choice(['a', 'b', 'c'], p=[pct_case_a, pct_case_b, pct_case_c])

        # 本人が被保険者
        if case_var == 'a':
            insured_person = contractor
            insured_address = contractor_address

        # 家族が被保険者
        elif case_var == 'b':
            #　家族の情報をランダムに選択し、姓には契約者の情報をセットする
            family = df_person.iloc[person_index[contract_index]].copy()
            family['last_name'] = contractor.last_name
            family['last_name_pronunciation'] = contractor.last_name_pronunciation

            # 家族が被保険者の場合は30%の確立で学資保険に変更
            if random.random() >= 0.7:
                contract_dict['type'] = '学資'
                family['age'] = int(random.random() * 12)
            
            insured_person = family
            insured_address = contractor_address

            
        # それ以外が被保険者
        else:
            insured_person = df_person.iloc[person_index[contract_index]]
            insured_address = df_addr.iloc[addr_index[contract_index]]

        # 設定された被保険者情報を契約dictにセットする
        contract_dict = set_person_info('insured', insured_person, contract_dict)
        contract_dict = set_address('insured', insured_address, contract_dict)

        # 生成した契約情報のlistにセット
        contracts.append(copy.copy(contract_dict))
        contract_index += 1
    
    
df_contract = pd.DataFrame(contracts)
print('finished')

finished


In [142]:
# 契約の一覧を生成
person_index = [random.randint(0,df_person.shape[0]) for x in range(0,num_contractors * max_contracts)]
addr_index = [random.randint(0,df_addr.shape[0]) for x in range(0,num_contractors * max_contracts)]

contracts = []
contract_index = 0
for i in range(0, num_contractors):
    # 保険契約者の個人情報、住所情報をランダムに選択する
    contract_dict = {}
    contractor = df_person.sample(n=1).iloc[0]
    contractor_address = df_addr.sample(n=1).iloc[0]

    # 受理者CDをセット
    contract_dict['staff_cd'] = random.randint(0, num_staffs - 1)
    
    # 契約情報をセット
    contract_dict = set_person_info('contractor', contractor, contract_dict)
    contract_dict = set_address('contractor', contractor_address, contract_dict)
        
    # 一人の契約者が保持する契約の数をランダムに決める
    num_contract = int(random.random()  ** 2 * max_contracts) + 1

    # 契約の属性情報を決定する
    cases = np.random.choice(['a', 'b', 'c'], num_contract, p=[pct_case_a, pct_case_b, pct_case_c])
    contract_types = np.random.choice(['終身', '定期', '養老', '長寿支援', '特別終身'], num_contract, p = [0.4, 0.2, 0.2, 0.1, 0.1])
    insured_candidates = df_person.sample(n=num_contract)
    candidate_addresses = df_addr.sample(n=num_contract)
    
    # 契約の数分ループを回す
    for case_var, row, contract_type in zip(cases, insured_candidates.iterrows(), contract_types):
        _, insured_candidate = row
        
        # 保険契約の種類をセット
        contract_dict['type'] = contract_type
        
        # 被保険者情報を3つのケースから選択する
        # 本人が被保険者
        if case_var == 'a':
            insured_person = contractor
            insured_address = contractor_address

        # 家族が被保険者
        elif case_var == 'b':
            #　家族の情報をランダムに選択し、姓には契約者の情報をセットする
            insured_candidate['last_name'] = contractor.last_name
            insured_candidate['last_name_pronunciation'] = contractor.last_name_pronunciation

            # 家族が被保険者の場合は30%の確立で学資保険に変更
            if random.random() >= 0.7:
                contract_dict['type'] = '学資'
                insured_candidate['age'] = int(random.random() * 12)
            
            insured_person = insured_candidate
            insured_address = contractor_address
            
        # それ以外が被保険者
        else:
            insured_person = insured_candidate
            insured_address = df_addr.iloc[addr_index[contract_index]]

        # 設定された被保険者情報を契約dictにセットする
        contract_dict = set_person_info('insured', insured_person, contract_dict)
        contract_dict = set_address('insured', insured_address, contract_dict)

        # 生成した契約情報のlistにセット
        contracts.append(copy.copy(contract_dict))
        contract_index += 1
            
df_contract = pd.DataFrame(contracts)
print('finished')

finished


In [145]:
df_contract.head(20)

,contractor_address_1,contractor_address_2,contractor_address_3,contractor_age,contractor_birthday,contractor_first_name,contractor_first_name_pron,contractor_gender,contractor_last_name,contractor_last_name_pron,contractor_phone,contractor_postal_cd,insured_address_1,insured_address_2,insured_address_3,insured_age,insured_birthday,insured_first_name,insured_first_name_pron,insured_gender,insured_last_name,insured_last_name_pron,insured_phone,insured_postal_cd,staff_cd,type
0,東京都,豊島区,南大塚２丁目２９番７号,15,2004/8/25,実桜,ミオ,女,江崎,エサキ,090-8967-9720,170-8457,東京都,豊島区,南大塚２丁目２９番７号,15,2004/8/25,実桜,ミオ,女,江崎,エサキ,090-8967-9720,170-8457,17,定期
1,東京都,豊島区,南大塚２丁目２９番７号,15,2004/8/25,実桜,ミオ,女,江崎,エサキ,090-8967-9720,170-8457,東京都,豊島区,南大塚２丁目２９番７号,15,2004/8/25,実桜,ミオ,女,江崎,エサキ,090-8967-9720,170-8457,17,養老
2,東京都,豊島区,南大塚２丁目２９番７号,15,2004/8/25,実桜,ミオ,女,江崎,エサキ,090-8967-9720,170-8457,東京都,豊島区,南大塚２丁目２９番７号,0,1987/3/24,裕子,ヒロコ,女,江崎,エサキ,080-6117-0321,170-8457,17,学資
3,東京都,豊島区,南大塚２丁目２９番７号,15,2004/8/25,実桜,ミオ,女,江崎,エサキ,090-8967-9720,170-8457,東京都,豊島区,南大塚２丁目２９番７号,15,2004/8/25,実桜,ミオ,女,江崎,エサキ,090-8967-9720,170-8457,17,定期
4,東京都,豊島区,南大塚２丁目２９番７号,15,2004/8/25,実桜,ミオ,女,江崎,エサキ,090-8967-9720,170-8457,東京都,豊島区,南大塚２丁目２９番７号,15,2004/8/25,実桜,ミオ,女,江崎,エサキ,090-8967-9720,170-8457,17,定期
5,東京都,目黒区,青葉台３丁目６−３０,30,1989/7/14,好子,ヨシコ,女,斉藤,サイトウ,090-3312-5169,153-8523,東京都,目黒区,青葉台３丁目６−３０,30,1989/7/14,好子,ヨシコ,女,斉藤,サイトウ,090-3312-5169,153-8523,19,終身
6,東京都,千代田区,内幸町２丁目１−４,48,1971/7/1,翔平,ショウヘイ,男,保科,ホシナ,090-9004-7319,100-8507,東京都,千代田区,内幸町２丁目１−４,65,1954/3/20,幹雄,ミキオ,男,保科,ホシナ,080-3179-5167,100-8507,16,終身
7,東京都,千代田区,内幸町２丁目１−４,48,1971/7/1,翔平,ショウヘイ,男,保科,ホシナ,090-9004-7319,100-8507,東京都,青梅市,末広町１丁目７−２,51,1968/5/3,哲美,テツミ,女,高倉,タカクラ,080-1392-4221,198-8711,16,終身
8,東京都,千代田区,内幸町２丁目１−４,48,1971/7/1,翔平,ショウヘイ,男,保科,ホシナ,090-9004-7319,100-8507,東京都,千代田区,内幸町２丁目１−４,26,1993/3/4,光代,ミツヨ,男,保科,ホシナ,090-8672-0042,100-8507,16,特別終身
9,東京都,江東区,亀戸１丁目５−７日鐵ＮＤタワー,50,1969/11/3,百香,モモカ,女,青柳,アオヤギ,090-2591-8915,136-8555,東京都,江東区,亀戸１丁目５−７日鐵ＮＤタワー,25,1994/10/3,真実,マミ,女,青柳,アオヤギ,080-2189-6035,136-8555,15,終身


## 生成したデータをcsvに出力

In [146]:
ordered_columns = [
 'staff_cd',
 'type',
 'contractor_last_name',
 'contractor_last_name_pron',
 'contractor_first_name',
 'contractor_first_name_pron',
 'contractor_age',
 'contractor_birthday',
 'contractor_gender',
 'contractor_postal_cd',
 'contractor_address_1',
 'contractor_address_2',
 'contractor_address_3',
 'contractor_phone',
 'insured_last_name',
 'insured_last_name_pron',
 'insured_first_name',
 'insured_first_name_pron',
 'insured_age',
 'insured_birthday',
 'insured_gender',
 'insured_postal_cd',
 'insured_address_1',
 'insured_address_2',
 'insured_address_3',
 'insured_phone'
 ]

In [147]:
df_staff.to_csv('data/generated_staffs.csv', index=False)
df_contract[ordered_columns].to_csv('data/generated_contracts.csv', index=False)

### （オプション）出力したcsvファイルを読み込んで表示

In [14]:
df_generated_contract = pd.read_csv('data/generated_contracts.csv')

In [15]:
pd.set_option("display.max_colwidth", 80)
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 100)

df_generated_contract.head(20)

,staff_cd,contractor_last_name,contractor_last_name_pron,contractor_first_name,contractor_first_name_pron,contractor_age,contractor_birthday,contractor_gender,contractor_postal_cd,contractor_address_1,contractor_address_2,contractor_address_3,contractor_phone,insured_last_name,insured_last_name_pron,insured_first_name,insured_first_name_pron,insured_age,insured_birthday,insured_gender,insured_postal_cd,insured_address_1,insured_address_2,insured_address_3,insured_phone
0,2,金川,カナガワ,柚季,ユズキ,21,1998/11/11,女,198-8700,東京都,青梅市,東青梅１丁目１１−１,080-1094-4158,衛藤,エトウ,邦夫,クニオ,25,1994/10/20,男,135-8481,東京都,江東区,永代１丁目１３−３,090-6924-9966
1,11,笹井,ササイ,優斗,ユウト,26,1993/6/29,男,106-8550,東京都,港区,六本木６丁目２−３１,090-8091-5387,笹井,ササイ,優斗,ユウト,26,1993/6/29,男,106-8550,東京都,港区,六本木６丁目２−３１,090-8091-5387
2,8,藤野,フジノ,善一,ゼンイチ,50,1969/9/5,男,110-8560,東京都,台東区,台東１丁目５−１,090-0275-2422,藤野,フジノ,善一,ゼンイチ,50,1969/9/5,男,110-8560,東京都,台東区,台東１丁目５−１,090-0275-2422
3,8,藤野,フジノ,善一,ゼンイチ,50,1969/9/5,男,110-8560,東京都,台東区,台東１丁目５−１,090-0275-2422,藤野,フジノ,善一,ゼンイチ,50,1969/9/5,男,110-8560,東京都,台東区,台東１丁目５−１,090-0275-2422
4,3,倉本,クラモト,遙香,ハルカ,35,1984/3/4,女,115-8555,東京都,北区,赤羽南２丁目５−１,080-0037-0485,倉本,クラモト,遙香,ハルカ,35,1984/3/4,女,115-8555,東京都,北区,赤羽南２丁目５−１,080-0037-0485
5,11,平田,ヒラタ,晃,アキラ,32,1987/4/28,男,130-8655,東京都,墨田区,本所３丁目６−６,090-4382-4957,平田,ヒラタ,晃,アキラ,32,1987/4/28,男,130-8655,東京都,墨田区,本所３丁目６−６,090-4382-4957
6,8,小貫,コヌキ,貞次,テイジ,19,2000/6/23,男,153-8556,東京都,目黒区,目黒２丁目１３−１８,080-9739-6469,小貫,コヌキ,貞次,テイジ,19,2000/6/23,男,153-8556,東京都,目黒区,目黒２丁目１３−１８,080-9739-6469
7,8,大町,オオマチ,萌恵,モエ,44,1975/11/21,女,161-8560,東京都,新宿区,西落合１丁目３１−４,080-0472-0156,大町,オオマチ,清佳,サヤカ,23,1996/7/26,女,161-8560,東京都,新宿区,西落合１丁目３１−４,080-8834-1668
8,20,影山,カゲヤマ,峻輝,シュンキ,38,1981/3/12,男,186-8519,東京都,国立市,中１丁目８−５,080-1945-1763,影山,カゲヤマ,峻輝,シュンキ,38,1981/3/12,男,186-8519,東京都,国立市,中１丁目８−５,080-1945-1763
9,15,杉原,スギハラ,重信,シゲノブ,30,1989/3/21,男,108-8430,東京都,港区,海岸３丁目１８−２１ブライトイースト芝浦,090-6610-4676,杉原,スギハラ,紗良,サラ,47,1972/3/13,女,108-8430,東京都,港区,海岸３丁目１８−２１ブライトイースト芝浦,080-6456-2349


In [19]:
df_generated_contract.iloc[0]

staff_cd                                  2
contractor_last_name                     金川
contractor_last_name_pron              カナガワ
contractor_first_name                    柚季
contractor_first_name_pron              ユズキ
contractor_age                           21
contractor_birthday              1998/11/11
contractor_gender                         女
contractor_postal_cd               198-8700
contractor_address_1                    東京都
contractor_address_2                    青梅市
contractor_address_3             東青梅１丁目１１−１
contractor_phone              080-1094-4158
insured_last_name                        衛藤
insured_last_name_pron                  エトウ
insured_first_name                       邦夫
insured_first_name_pron                 クニオ
insured_age                              25
insured_birthday                 1994/10/20
insured_gender                            男
insured_postal_cd                  135-8481
insured_address_1                       東京都
insured_address_2               